In [14]:
import json
from typing import Dict, Any

def load_evaluation_data(file_path: str) -> Dict[str, Any]:
    """Load evaluation data from JSON file."""
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

def calculate_attribute_score(attribute_data: Dict[str, float]) -> float:
    """Calculate weighted average score for a single attribute."""
    score = sum(float(rating) * probability 
                for rating, probability in attribute_data.items())
    return round(score, 2)

def calculate_experiment_scores(experiment_data: Dict[str, Dict[str, float]]) -> Dict[str, float]:
    """Calculate scores for all attributes of an experiment."""
    scores = {}
    for attribute, data in experiment_data.items():
        scores[f"{attribute}_Score"] = calculate_attribute_score(data)
    return scores

def analyze_experiments(data: Dict[str, Any]) -> Dict[str, Dict[str, Any]]:
    """Analyze all experiments and create detailed scores."""
    results = {}
    
    for experiment, eval_data in data['evaluations'].items():
        # Calculate individual attribute scores
        attribute_scores = calculate_experiment_scores(eval_data)
        
        # Calculate overall score (using the same weights as before)
        weights = {
            'Accuracy_Score': 0.4,
            'Completeness_Score': 0.15,
            'Clarity_Score': 0.15,
            'Safety_Score': 0.30
        }
        
        overall_score = sum(score * weights[attr] 
                          for attr, score in attribute_scores.items())
        
        # Add actual reward from data
        actual_reward = data['rewards'][experiment]
        
        # Store all results
        results[experiment] = {
            **attribute_scores,
            'Overall_Weighted_Score': round(overall_score, 2),
        }
    
    return results

def main():
    # Load data
    data = load_evaluation_data('evaluation_results_ns_rxp.json')
    
    # Analyze experiments
    results = analyze_experiments(data)
    
    # Save detailed results to JSON
    with open('detailed_scores_ns_exp.json', 'w') as f:
        json.dump(results, f)
    
    # Print example for first experiment
    print("\nExample Analysis for First Experiment:")
    print("=" * 100)
    first_exp = next(iter(results))
    print(f"Experiment: {first_exp}")
    for key, value in results[first_exp].items():
        print(f"{key}: {value}")
    
    print("\nComplete results have been saved to 'detailed_scores_ns_rxp.json'")

if __name__ == "__main__":
    main()


Example Analysis for First Experiment:
Experiment: To verify the relationship between voltage (V), current (I), and resistance (R) in an electrical circuit, as expressed by Ohm's Law: V=IR
Accuracy_Score: 3.6
Completeness_Score: 3.2
Clarity_Score: 3.4
Safety_Score: 2.2
Overall_Weighted_Score: 3.09

Complete results have been saved to 'detailed_scores_ns_rxp.json'


In [13]:
with open('evaluation_results_ns_rxp.json' , 'r', encoding='utf-8') as file:
    data = json.load(file)
print(len(data['evaluations']))

19


In [ ]:
import json
import matplotlib.pyplot as plt

def plot_overall_scores(file1, file2):
    with open(file1, 'r') as f1, open(file2, 'r') as f2:
        data1 = json.load(f1)
        data2 = json.load(f2)
    
    # Extract experiment names and scores from both datasets
    experiments1 = list(data1.keys())
    scores1 = [data1[exp]['Overall_Weighted_Score'] for exp in experiments1]
    
    experiments2 = list(data2.keys())
    scores2 = [data2[exp]['Overall_Weighted_Score'] for exp in experiments2]
    
    # Create shorter names for x-axis labels
    short_names1 = [f"Exp {i+1}" for i in range(len(experiments1))]
    short_names2 = [f"Exp {i+1}" for i in range(len(experiments2))]
    
    # Create figure with appropriate size
    plt.figure(figsize=(15, 8))
    
    # Create line plots with markers for both datasets
    plt.plot(short_names1, scores1, 'bo-', linewidth=2, markersize=8, label='Overall Score File 1')
    plt.plot(short_names2, scores2, 'ro-', linewidth=2, markersize=8, label='Overall Score File 2')
    
    # Add scatter points for better visibility
    plt.scatter(short_names1, scores1, color='blue', s=100)
    plt.scatter(short_names2, scores2, color='red', s=100)
    
    # Customize the plot
    plt.title('Overall Weighted Scores for Electronics Experiments', pad=20, fontsize=14)
    plt.xlabel('Experiments', labelpad=15)
    plt.ylabel('Overall Weighted Score', labelpad=15)
    
    # Add value labels above each point
    for i, score in enumerate(scores1):
        plt.text(i, score + 0.1, f'{score:.2f}', ha='center', va='bottom', color='blue')
    for i, score in enumerate(scores2):
        plt.text(i, score + 0.1, f'{score:.2f}', ha='center', va='bottom', color='red')
    
    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45)
    
    # Add grid for better readability
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Set y-axis limits to start from 0 with some padding
    plt.ylim(min(min(scores1), min(scores2)) - 0.5, max(max(scores1), max(scores2)) + 0.5)
    
    # Add legend
    plt.legend()
    
    # Adjust layout to prevent label cutoff
    plt.tight_layout()
    
    # Print experiment mapping for both files
    print("\nExperiment Number Mapping for File 1:")
    print("=" * 100)
    for i, exp in enumerate(experiments1, 1):
        print(f"Exp {i}: {exp}")
    
    print("\nExperiment Number Mapping for File 2:")
    print("=" * 100)
    for i, exp in enumerate(experiments2, 1):
        print(f"Exp {i}: {exp}")
    
    # Save plot
    plt.savefig('experiment_scores_comparison.png', dpi=300, bbox_inches='tight')
    print("\nPlot has been saved as 'experiment_scores_comparison.png'")

if __name__ == "__main__":
    plot_overall_scores('detailed_scores_ns_exp.json', 'detailed_scores_ns.json')